<a href="https://colab.research.google.com/github/pikey-msc/RiesgosFinancieros/blob/master/2020-1/Insumos/Bono_M.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%load_ext rpy2.ipython
!rm -rf RiesgosFinancieros
!git clone "https://github.com/pikey-msc/RiesgosFinancieros/"

Cloning into 'RiesgosFinancieros'...
remote: Enumerating objects: 42, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 42 (delta 7), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (42/42), done.


Características

Valor nominal: Depende emisora, $100$ MXN (Bonos M), $1,000$ USD (United Mexican States: UMS).

Plazo: Depende emisora, 3, 5, 7, 10, 20 y 30 años (Bonos M), y 5, 10, 15, 20, 25 y 30 años (UMS).

Colocación: Subasta de Banco de México.

Negociación: Depende emisora, a tasa de rendimiento (Bonos M), a tasa de rendimiento y en USD (UMS)

Intereses: Depende de emisora, 182 días (Bonos M), 6 meses o anual (la tasa puede ser revisable). 


In [0]:
%%R

btasadesc="RiesgosFinancieros/2020-1/Insumos/tasa_yield.txt"
tfcupon=cbind( 0.065,	0.0675,	0.07,	0.075,	0.078) #Tasafija del cupón
plazos=cbind( 378,	405,	550,	1200,	1800) #Vencimiento del bono
plazocupon=cbind( 182,	182, 182, 182, 182) #plazos fijos de cada cupón
contratos=cbind(22000, -29000, 29000, -46000, 10000) #posición invertida
nominal=100
itpl=1 #poner 0 si se quiere interpolación lineal o 1 si se quiere tasa alambrada


In [0]:
%%R

#carga de tasas de descuento
data1<-read.table(btasadesc)
n<-nrow(data1)
m_orig=ncol(data1)
X_orig=data.frame(data1[2:n,2:m_orig])
nodos=data.frame(data1[1,2:m_orig])
n=n-1

In [0]:
%%R
#Tasa alambrada
talamb=function(nodos,curva,plazos) #función de interpolación de tasas por el método alamabrada
{
  n=max(ncol(plazos),1)
  m=max(ncol(nodos),1)
  TC=matrix(0,1,n)
  TL=matrix(0,1,n)
  TF=matrix(0,1,n)
  for (j in 1:n)
  {
    i=1
    repeat
    {
      if(nodos[i]<= plazos[j] && plazos[j] <=nodos[i+1])
      {
        TC[j]=curva[i]
        TL[j]=curva[i+1]
        TF[j]=((((1+TL[j]*nodos[i+1]/360)/(1+TC[j]*nodos[i]/360))^((plazos[j]-nodos[i])/(nodos[i+1]-nodos[i]))*(1+TC[j]*nodos[i]/360))-1)*360/plazos[j]
        break
      }
      else if (plazos[j]<nodos[1])
      {
        TC[j]=curva[1]
        TL[j]=curva[1]
        TF[j]=curva[1]
        break
      }
      else if (plazos[j]>nodos[m])
      {
        TC[j]=curva[m]
        TL[j]=curva[m]
        TF[j]=curva[m]
        break
      }
      else
      {i=i+1}
    }
  }
  as.matrix(t(as.numeric(rbind(TF))))
}

In [0]:
%%R
m=ncol(plazos)
X=matrix(0,n-1,m)
for (i in 1:(n-1))
{
X[i,]=if(itpl==0){approx(nodos,X_orig[i,],plazos)$y}else{talamb(nodos,X_orig[i,],plazos)}
}

$$V_{ty}=\sum_{i=1}^{n}\frac{N\cdot C \cdot t_{c}\cdot p_c/360}{(1+t_{n} \cdot p_c/360)^{(p_i/p_c)}} + \frac{N\cdot C}{(1+t_{n} \cdot p_c/360)^{(p_n/p_c)}} \quad \textrm{ó}$$
	$$ V_{ty}=\Bigg(\frac{N\cdot C \cdot t_{c}\cdot p_c}{360}\bigg( \frac{(1-(1+\frac{t_n\cdot p_c}{360})^{-(\Big[\frac{p_n}{p_c}\Big]+1)})}{t_n\cdot p_c/360}\bigg) + \frac{N\cdot C}{(1+\frac{t_{n} \cdot p_c}{360})^{(\Big[\frac{p_n}{p_c}\Big]+1)}}\Bigg)\bigg(1+\frac{t_n\cdot p_c}{360}\bigg)^{1-\frac{p_1}{p_c}}$$			
	
Donde $V_{ty}$: Valor del bono con tasa yield $t_n$ (Precio Sucio)
$t_n$: tasa de interés al vencimiento.
$N$: Valor Nominal del bono
$C$: Número de contratos
$p_{c}$: Plazo fijo para cada pago de intereses del cupón
$p_{i}$: Plazo acumulado (en días) al cupón $i$
$t_{c}$: Tasa cupón fija. 
En este caso se tiene un factor de riesgo subyacente y UN SÓLO factor de riesgo que son los cupones. 


In [8]:
%%R
bonoMyield=function(x, plazos, plazocupon, tfcupon, nominal, contratos) #valoración bono tasa fija
{
x=x0  
N=as.integer(plazos/plazocupon)+1   
a=(1-(1+x*plazocupon/360)^(-N))/(plazocupon*x/360)
p1=plazos-plazocupon*(N-1)   
((contratos*nominal*tfcupon*plazocupon/360)*a+(contratos*nominal)/((1+x*plazocupon/360)^N))*(1+x*plazocupon/360)^(1-p1/plazocupon)   
}

x0=X[1,] #tasas de descuento valor actual
bonoMyield(x0,plazos, plazocupon, tfcupon, nominal, contratos)



        [,1]     [,2]    [,3]     [,4]    [,5]
[1,] 2236687 -2943124 2971369 -4645964 1009444


In [10]:
%%R
#OTRA OPCIÓN DE HACER LA VALORACIÓN
bonotasafija=function(x, plazos, plazocupon, tfcupon, nominal, contratos) #valoración bono tasa fija
{
	m=max(ncol(plazos),1)
	N=matrix(0,1,m)
	for (j in (1:m))
	{
	N[j]=as.integer(plazos[j]/plazocupon[j])+1
	}
	plazini=plazos-plazocupon*(N-1)
	svp=matrix(0,1,m)
	for (j in (1:m))
	{
	vp=matrix(0,1,N[j])
		for (i in (1:N[j]))
		{
		if(i==N[j])
			{
			plazocupcurr=plazini[j]+plazocupon[j]*(i-1)
			vp[i]=(contratos[j]*nominal*tfcupon[j]*plazocupon[j]/360+contratos[j]*nominal)*(1+x[j]*plazocupon[j]/360)^(-(plazocupcurr/plazocupon[j]))
			svp[j]=vp[i]+svp[j]
			}
		else
			{
			plazocupcurr=plazini[j]+plazocupon[j]*(i-1)
			vp[i]=contratos[j]*nominal*tfcupon[j]*plazocupon[j]/360*(1+x[j]*plazocupon[j]/360)^(-(plazocupcurr/plazocupon[j]))
			svp[j]=vp[i]+svp[j]
			}
		}
	}
		svp
}


V0=bonotasafija(x0, plazos, plazocupon, tfcupon, nominal, contratos) #Valor del portafolio
V0

        [,1]     [,2]    [,3]     [,4]    [,5]
[1,] 2236687 -2943124 2971369 -4645964 1009444
